In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2023-01-01 00:34:00.441808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-01 00:34:02.277389: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-01 00:34:02.277491: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-01 00:34:06.997994: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
!pip install kneed

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
df = pd.read_csv('./datasets/Reuters.csv', dtype="str")
df

,id,text,label
0,10,COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...,acq
1,1000,NATIONAL AMUSEMENTS AGAIN UPS VIACOM &lt;VIA> ...,acq
2,10000,ROGERS &lt;ROG> SEES 1ST QTR NET UP SIGNIFICAN...,earn
3,10002,ISLAND TELEPHONE SHARE SPLIT APPROVED\n &lt;I...,earn
4,10008,QUESTECH INC &lt;QTEC> YEAR NET\n Shr loss ni...,earn
...,...,...,...
5496,9988,FED SETS TWO BILLION DLR CUSTOMER REPURCHASE\n...,interest
5497,9992,KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...,earn
5498,9993,TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...,earn
5499,9994,NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...,earn


# Preprocessing

In [5]:
X_train = df["text"]
labels = df["label"]
print(X_train.shape,labels.shape)

(5501,) (5501,)


In [6]:
X_train

0       COMPUTER TERMINAL SYSTEMS &lt;CPML> COMPLETES ...
1       NATIONAL AMUSEMENTS AGAIN UPS VIACOM &lt;VIA> ...
2       ROGERS &lt;ROG> SEES 1ST QTR NET UP SIGNIFICAN...
3       ISLAND TELEPHONE SHARE SPLIT APPROVED\n  &lt;I...
4       QUESTECH INC &lt;QTEC> YEAR NET\n  Shr loss ni...
                              ...                        
5496    FED SETS TWO BILLION DLR CUSTOMER REPURCHASE\n...
5497    KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n  Q...
5498    TECHNITROL INC &lt;TNL> SETS QUARTERLY\n  Qtly...
5499    NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...
5500    &lt;A.H.A. AUTOMOTIVE TECHNOLOGIES CORP> YEAR ...
Name: text, Length: 5501, dtype: object

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(labels)
labels_encoded = le.transform(labels)

## Keras Vectorization

In [8]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
tokenized_X = [word_tokenize(sample) for  sample in X_train]

[nltk_data] Downloading package stopwords to /home/vavre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/vavre/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
keras_tokenized_nostopwords_X = tokenized_X.copy()
ps = PorterStemmer()
for idx, sentence in enumerate(tokenized_X):
  keras_tokenized_nostopwords_X[idx] = [ps.stem(word) for word in sentence if not word.lower() in stop_words]
  print(f"{ len(tokenized_X[idx]) - len(keras_tokenized_nostopwords_X[idx])} words removed")
  print("============")

72 words removed
44 words removed
37 words removed
5 words removed
2 words removed
24 words removed
7 words removed
31 words removed
1 words removed
6 words removed
0 words removed
2 words removed
0 words removed
0 words removed
41 words removed
12 words removed
9 words removed
34 words removed
125 words removed
2 words removed
15 words removed
1 words removed
38 words removed
24 words removed
11 words removed
0 words removed
0 words removed
6 words removed
31 words removed
34 words removed
17 words removed
4 words removed
50 words removed
0 words removed
1 words removed
272 words removed
6 words removed
3 words removed
14 words removed
11 words removed
7 words removed
0 words removed
24 words removed
0 words removed
5 words removed
56 words removed
17 words removed
7 words removed
21 words removed
4 words removed
1 words removed
21 words removed
32 words removed
1 words removed
1 words removed
22 words removed
56 words removed
10 words removed
39 words removed
9 words removed
1 words 

In [10]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from tensorflow.keras.layers import TextVectorization

detokenizer = TreebankWordDetokenizer()
keras_detokenized_nostopwords_X = [ detokenizer.detokenize(sample) for sample in keras_tokenized_nostopwords_X]


In [11]:
from tensorflow.keras.layers import TextVectorization

keras_vectorizer = TextVectorization( output_mode="tf-idf")
text_ds = tf.data.Dataset.from_tensor_slices(keras_detokenized_nostopwords_X).batch(128)
keras_vectorizer.adapt(text_ds)
keras_X_train_vectorized = keras_vectorizer(keras_detokenized_nostopwords_X)
keras_X_train_vectorized.shape

2023-01-01 00:41:29.748349: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-01 00:41:29.748806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-01 00:41:29.749104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-01 00:41:29.749395: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-01 00:41:29.749577: W tensorf

TensorShape([5501, 27073])

In [12]:
keras_X_train_vectorized

<tf.Tensor: shape=(5501, 27073), dtype=float32, numpy=
array([[0.       , 7.1303625, 1.114995 , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 3.0558696, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 4.074493 , 1.114995 , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 3.344985 , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 2.22999  , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)>

## SKLearn Vectorization

In [13]:
def text_preprocess_analyzer(sample):
  tokens = word_tokenize(sample)
  tokens = [ps.stem(token) for token in tokens if not token.lower() in stop_words]
  return tokens

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
sklearn_vectorizer = TfidfVectorizer(analyzer=text_preprocess_analyzer)
sklearn_X_train_vectorized = sklearn_vectorizer.fit_transform(X_train)
sklearn_X_train_vectorized.shape

(5501, 28069)

In [15]:
sklearn_X_train_vectorized

<5501x28069 sparse matrix of type '<class 'numpy.float64'>'
	with 303588 stored elements in Compressed Sparse Row format>

# Clustering

In [16]:
import numpy as np
from sklearn import metrics

def purity_score(y_true, y_pred):
    # compute confusion matrix
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 


## Clustering with Keras KMeans

In [17]:
X_train_used = keras_X_train_vectorized

In [18]:
from sklearn.cluster import KMeans
rss = []
for k in range(1, 11):
  kmeans = KMeans(n_clusters=k, max_iter=2)
  kmeans.fit(X_train_used)
  rss.append(kmeans.inertia_)

/home/vavre/information-retrieval/information-retrieval-ut/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/vavre/information-retrieval/information-retrieval-ut/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/vavre/information-retrieval/information-retrieval-ut/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/vavre/information-retrieval/information-retrieval-ut/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: Th

In [19]:
import matplotlib.pyplot as plt 
from kneed import KneeLocator

plt.plot(range(1, 11), rss)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("RSS")
plt.show()

keras_kl = KneeLocator(range(1, 11), rss, direction="decreasing", curve="convex")
print(f'KneeLocator Elbow is: {keras_kl.elbow}')

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
keras_kmeans = KMeans(n_clusters=keras_kl.elbow)
keras_clusters = keras_kmeans.fit(X_train_used)

In [ ]:
np.unique(keras_clusters.labels_)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score
keras_purity = purity_score(labels_encoded,keras_clusters.labels_ )
keras_nmi = normalized_mutual_info_score(labels_encoded,keras_clusters.labels_)

# 0.5289947282312307
print(f"purity = {keras_purity} , nmi = {keras_nmi}")

## Clustering with SKLearn KMeans

In [ ]:
X_train_used = sklearn_X_train_vectorized

In [ ]:
from sklearn.cluster import KMeans
rss = []
for k in range(1, 11):
  kmeans = KMeans(n_clusters=k, max_iter=800)
  kmeans.fit(X_train_used)
  rss.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt 
from kneed import KneeLocator

plt.plot(range(1, 11), rss)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("RSS")
plt.show()

sklearn_kl = KneeLocator(range(1, 11), rss, direction="decreasing", curve="convex")
print(f'KneeLocator Elbow is: {sklearn_kl.elbow}')

In [ ]:
sklearn_kmeans = KMeans(n_clusters=sklearn_kl.elbow)
sklearn_clusters = sklearn_kmeans.fit(X_train_used)

In [ ]:
np.unique(sklearn_clusters.labels_)

In [ ]:
from sklearn.metrics import normalized_mutual_info_score
sklearn_purity = purity_score(labels_encoded,sklearn_clusters.labels_ )
sklearn_nmi = normalized_mutual_info_score(labels_encoded,sklearn_clusters.labels_)
print(f"purity = {sklearn_purity} , nmi = {sklearn_nmi}")

## Keras and SKlearn Conclusion

In [ ]:
print(f"For Keras  ==>  Purity = {keras_purity} , NMI = {keras_nmi}")
print(f"For Sklearn  ==>  Purity = {sklearn_purity} , NMI = {sklearn_nmi}")

## Clustering AgglomerativeClustering 

In [ ]:
from sklearn.cluster import AgglomerativeClustering


# ## Single Link
# single_clusters = AgglomerativeClustering(n_clusters=sklearn_kl.elbow, linkage="single").fit(sklearn_X_train_vectorized.toarray())
# ## Complete Link
# complete_clusters = AgglomerativeClustering(n_clusters=sklearn_kl.elbow, linkage="complete").fit(sklearn_X_train_vectorized.toarray())
# ## Average Link
# average_clusters = AgglomerativeClustering(n_clusters=sklearn_kl.elbow, linkage="average").fit(sklearn_X_train_vectorized.toarray())

# ## purities
# single_purity = purity_score(labels_encoded,single_clusters.labels_ )
# complete_purity = purity_score(labels_encoded,complete_clusters.labels_ )
# average_purity = purity_score(labels_encoded,average_clusters.labels_ )

# ## nmi
# single_nmi = normalized_mutual_info_score(labels_encoded,single_clusters.labels_)
# complete_nmi = normalized_mutual_info_score(labels_encoded,complete_clusters.labels_)
# average_nmi = normalized_mutual_info_score(labels_encoded,average_clusters.labels_)

# ## conclusions
# print(f"For Single link  ==>  Purity = {single_purity} , NMI = {single_nmi}")
# print(f"For Complete link  ==>  Purity = {complete_purity} , NMI = {complete_nmi}")
# print(f"For Average link  ==>  Purity = {average_purity} , NMI = {average_nmi}")
# print(f"For Keras KMeans  ==>  Purity = {keras_purity} , NMI = {keras_nmi}")
# print(f"For Sklearn KMeans  ==>  Purity = {sklearn_purity} , NMI = {sklearn_nmi}")

In [ ]:
single_clusters = AgglomerativeClustering(n_clusters=sklearn_kl.elbow, linkage="single").fit(sklearn_X_train_vectorized.toarray())
